In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import time
from bs4 import BeautifulSoup
import twint
import nest_asyncio
nest_asyncio.apply()

### Webscraping

In [2]:
url = "https://triagecancer.org/congressional-social-media"
soup = BeautifulSoup(requests.get(url).text)

In [3]:
containers = [soup.findAll('tr', {"class": f"ninja_table_row_{num}"}) for num in range(541)]

In [4]:
names = [c[0].findAll("td")[2].text for c in containers]

In [5]:
parties = [c[0].findAll("td")[4].text for c in containers]

In [6]:
handles = [c[0].findAll("td")[5].text for c in containers]

In [15]:
twitter_df = pd.DataFrame(columns = ["name", "party", "t_handle"])
twitter_df["name"] = names
twitter_df["party"] = parties
twitter_df["t_handle"] = handles

In [16]:
twitter_df["t_handle"] = twitter_df["t_handle"].map(lambda x: x.strip("@"))

In [17]:
add_names = ["Biden, Joe", "Clinton, Hillary", "Obama, Barack", "Pence, Mike", "Pompeo, Mike"]
add_parties = ["D", "D", "D", "R", "R"]
add_handles = ["JoeBiden", "HillaryClinton", "BarackObama", "Mike_Pence", "mikepompeo"]
d = {'name': add_names, 'party': add_parties, 't_handle': add_handles}
add_df = pd.DataFrame(data=d)

In [18]:
twitter_df = twitter_df.append(add_df)

In [19]:
most_popular = twitter_df[twitter_df["t_handle"].isin(["HillaryClinton", "JoeBiden", 
                                                   "BarackObama", "SenSanders",
                                                  "Mike_Pence","KamalaHarris"])]

In [20]:
less_popular = twitter_df[~twitter_df["t_handle"].isin(["HillaryClinton", "JoeBiden", 
                                                   "BarackObama", "SenSanders",
                                                  "Mike_Pence","KamalaHarris"])]

### Extracting Data from Twint

In [21]:
def extract_twint(user, since, min_likes):
    c = twint.Config()
    c.Username = user
    c.Since = since
    c.Lang = "en"
    c.Min_likes = min_likes
    c.Popular_tweets = True
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    return df

In [22]:
def extract_all(df, since, min_likes):
    users = list(df["t_handle"])
    data = pd.DataFrame([])
    for u in users:
        try:
            u_df = extract_twint(u, since, min_likes)
            data = data.append(u_df)
        except ValueError:
            continue
    return data

In [24]:
tweets_less = extract_all(less_popular,'2016-01-01', 50)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


CRITICAL:root:twint.get:User:'user'


[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


CRITICAL:root:twint.get:User:'user'


[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

...

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping w

CRITICAL:root:twint.get:User:'user'


[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


CRITICAL:root:twint.get:User:'user'


[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [26]:
tweets_more = extract_all(most_popular,'2016-01-01', 500)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [30]:
tweets_less.shape

(394509, 38)

In [31]:
tweets_less = tweets_less[tweets_less["language"] == "en"]

In [35]:
tweets_all = tweets_less.append(tweets_more)

In [40]:
tweets_all = tweets_all[tweets_all["language"] == "en"]

In [41]:
tweets_all = tweets_all[["id", "date", "tweet", "language", "username", "nlikes", "hashtags"]]

In [49]:
tweets_all["characters"] = tweets_all["tweet"].map(lambda x: len(x))

In [57]:
tweets_all.to_csv("tweets_all.csv")

In [58]:
twitter_df.rename(columns={"t_handle": "username"}, inplace=True)

In [118]:
expanded_df = tweets_all.join(twitter_df.set_index("username"), on="username", how="left")

In [119]:
cols = ['id', 'name', 'username', 'date', 'tweet', 'characters', 'language', 'nlikes', 'hashtags','party']
expanded_df = expanded_df[cols]

In [120]:
expanded_df = expanded_df[expanded_df["characters"] > 50]

In [137]:
tweets_all["date"] = pd.to_datetime(tweets_all["date"])

In [145]:
tweets_all.drop_duplicates(subset="id", inplace=True)